# RDD
https://wikidocs.net/28377

## 저수준 API란
- 스파크에는 두 종류의 저수준 API가 있음
  1. <strong>분산 데이터 처리</strong>를 위한 RDD
  2. <strong>분산형 공유 변수</strong>를 배포하고 다루기 위한 API(브로드캐스트 변수와 어큐뮬레이터)

### 저수준 API는 언제 사용할까
- 구조적 API 위주로 사용하는 것이 좋으나, <strong>고수준 API에서 제공하지 않는 기능</strong>이 필요한 경우에 사용
  - 클러스터의 물리적 데이터의 배치를 아주 세밀하게 제어해야하는 상황
  - RDD를 사용해 개발된 기존 코드를 유지해야 하는 상황
  - 사용자가 정의한 공유 변수를 다뤄야 하는 상황
      - 14장에서 설명

- 스파크의 모든 워크로드는 <strong>저수준 기능을 사용하는 기초적인 형태로 컴파일</strong>되므로 이를 이해하는 것은 많은 도움이 될 수 있음
  - <strong>DataFrame 트랜스포메이션</strong>을 호출하면 다수의 <strong>RDD 트랜스포메이션</strong>으로 변환

### 저수준 API는 어떻게 사용할까
- <strong>SparkContext</strong>는 저수준 API 기능을 사용하기 위한 진입 지점
- 스파크 클러스터에서 연산을 수행하는데 필요한 도구인 SparkSession을 이용해 접근 가능
- 15장에서 설명

In [ ]:
spark.sparkContext

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## RDD 개요
- <strong>불변성</strong>을 가지며 <strong>병렬</strong>로 처리할 수 있는 <strong>파티셔닝된 레코드</strong>의 모음

- RDD의 레코드는 프로그래머가 선택하는 <strong>자바, 스칼라, 파이썬의 객체</strong>
  - cf) DataFrame의 레코드는 스키마를 알고, 필드로 구성된 구조화된 로우

- 이러한 객체에는 <strong>사용자가 원하는 포맷</strong>을 사용해 원하는 모든 데이터를 저장할 수 있음

- 모든 값을 다루거나 값 사이의 상호작용 과정은 반드시 <strong>수동으로 정의</strong>

- spark에서는 RDD레코드의 내부 구조를 파악할 수 없으므로 <strong>수작업으로 최적화</strong>
  - 필터 재정렬, 집계 등의 최적화 기법 직접 구현
  
- Dataset과 유사하지만 RDD는 구조화된 데이터 엔진을 사용해서 데이터를 다루지 않음
    - RDD와 Dataset 사이의 전환은 매우 쉬움

### RDD 유형
- 두 가지 타입의 RDD를 만들 수 있음
  1. <strong>제네릭</strong> RDD 타입
  2. <strong>키-값</strong> RDD 타입
  
- <strong>RDD의 주요 속성</strong>
  - 파티션의 목록
  - 각 조각을 연산하는 함수
  - 다른 RDD와의 의존성 목록
  - 부가적으로 키-값 RDD를 위한 Partitioner
      - Partitioner는 RDD를 사용하는 주된 이유 중 하나. 올바른 사용자 정의 Partitioner를 사용하면 성능과 안정성이 크게 향상됨. 13장 키-값 쌍 RDD에서 다룸.
  - 부가적으로 각 조각을 연산하기 위한 기본 위치 목록
  
- 이러한 속성은 사용자 프로그램을 <strong>스케줄링하고 실행하는 스파크의 모든 처리 방식을 결정</strong>

- 또한 RDD 역시 <strong>트랜스포메이션, 액션</strong> 제공
  - DataFrame과 Dataset의 트랜스포메이션, 액션과 동일한 방식으로 동작
    - 하지만 RDD에는 '로우'개념이 없으므로 구조적 API에서 제공하는 여러 함수를 사용하지 못하므로 수동으로 처리

- <strong>언어별 성능 차이</strong>
  - 스칼라, 자바는 비슷
  - 파이썬은 상당한 성능 저하가 발생
    - 파이썬으로 만들어진 사용자 정의 함수를 사용해 로우마다 적용하는 것과 동일한 것
    - 오버헤드 발생: 직렬화 -> 파이썬 프로세스에 전달 -> 처리 -> 다시 직렬화 -> JVM에 반환
    - 따라서 구조적 API를 사용하는 것이 좋음

### RDD는 언제 사용할까
- 정말 필요한 경우가 아니라면 수동으로 RDD를 생성하면 X
- DataFrame이 RDD보다 더 효율적이고 안정적이고 표현력이 좋음
- 물리적으로 분산된 데이터(자체적으로 구성한 데이터 파티셔닝)에 세부적인 제어가 필요할 때 RDD를 사용하는 것이 가장 적합

### Dataset과 RDD의 케이스 클래스
https://wikidocs.net/29773
- Dataset은 구조적 API가 제공하는 풍부한 기능과 최적화 기법을 제공한다는 점이 RDD와의 큰 차이점
- Dataset을 사용하면 JVM 데이터 타입과 스파크 데이터 타입 중 어떤 것을 쓸지 고민하지 않아도 됨
  - 모두 성능 동일

In [ ]:
words.countByValue()

Out[30]: defaultdict(int,
 {'Spark': 1,
 'The': 1,
 'Definitive': 1,
 'Guide': 1,
 ':': 1,
 'Big': 1,
 'Data': 1,
 'Processing': 1,
 'Made': 1,
 'Simple': 1})

In [ ]:
words.first()

Out[31]: 'Spark'

In [ ]:
spark.sparkContext.parallelize(range(1,20)).max()

Out[34]: 19

In [ ]:
spark.sparkContext.parallelize(range(1,20)).min()

Out[35]: 1

In [ ]:
words.take(5)

Out[36]: ['Spark', 'The', 'Definitive', 'Guide', ':']

In [ ]:
#정렬
words.takeOrdered(5)

Out[37]: [':', 'Big', 'Data', 'Definitive', 'Guide']

In [ ]:
#최상윗값
words.top(5)

Out[38]: ['The', 'Spark', 'Simple', 'Processing', 'Made']

In [ ]:
words.saveAsTextFile('/FileStore/tables/bookeTitle')

In [ ]:
words.saveAsSequenceFile('/')

In [ ]:
words.saveAsHadoopFile('/')